In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

import time
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [98]:
train_df = pd.read_csv("Data/input.csv") 
tanish_df = pd.read_csv("Data/input_tanish.csv")

#mappings = {1: 0, 2: 1, 3:2}

#train_df.label = [mappings[item] for item in train_df.label]
train_df

,Unnamed: 0,patient_id,sequence,label
0,0,Z8309420,Headache • Genital herpes simplex • Osteoarthr...,2
1,1,Z9132953,disease • Pain in both feet • Cigarette smoker...,1
2,2,Z15402812,Reactions • Rituximab Neutropenia Family Histo...,1
3,3,Z11702635,a chair without difficulty and had a normal ga...,2
4,4,Z6393821,History Family History Problem Relation Age of...,1
...,...,...,...,...
1295,1295,Z6362498,• Colon cancer Brother colon cancer; • Migrain...,1
1296,1296,Z6593900,clinic Allergies Allergen Reactions • Codeine ...,2
1297,1297,Z12421627,CA' • Breast cancer Paternal Aunt Breast Cance...,1
1298,1298,Z9132953,informed consent*See LMR for Comment FAMILY HI...,1


In [15]:
X = train_df["sequence"]
y = train_df["label"]

y_label = y.to_numpy()
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X,y,random_state=0,test_size=0.1, stratify=y_label)

y_train_valid_label = y_train_valid.to_numpy()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, random_state=0, test_size=0.1, stratify=y_train_valid_label)

In [16]:
print(len(X_train), type(X_train))
print(len(X_test))
print(len(X_valid))

1053 <class 'pandas.core.series.Series'>
130
117


In [5]:
print(X_train)

1082    y.o. male who presents to OSH (Whitten Hospita...
156     the patient. The family history is positive fo...
925     daily etoh- reports 1/2 gallon/day of vodka Fa...
419     a chair without difficulty and had a normal ga...
520     • Colon cancer Brother colon cancer; • Migrain...
                              ...                        
250     Progress Note: No chief complaint on file. Enc...
461     between 1-4 these days. 7 a.m. - 1 p.m. or 11-...
1131    Headache • Genital herpes simplex • Osteoarthr...
418     Date • Rosacea • Tinnitus • Hypercholesterolem...
164     branch block • Cataract • Nonexudative age-rel...
Name: sequence, Length: 1053, dtype: object


In [7]:
y_train
# print(y_train[y_train["label"] == 0].count())
# print(y_train[y_train["label"] == 1].count())
# print(y_train[y_train["label"] == 2].count())

1082    2
156     1
925     1
419     2
520     1
       ..
250     2
461     1
1131    2
418     1
164     2
Name: label, Length: 1053, dtype: int64

In [47]:
# y_test = y_test.to_frame()

In [48]:
# print(y_test[y_test["label"] == 0].count())
# print(y_test[y_test["label"] == 1].count())
# print(y_test[y_test["label"] == 2].count())

label    3
dtype: int64
label    67
dtype: int64
label    60
dtype: int64


In [49]:
# y_valid = y_valid.to_frame()

In [50]:
# print(y_valid[y_valid["label"] == 0].count())
# print(y_valid[y_valid["label"] == 1].count())
# print(y_valid[y_valid["label"] == 2].count())

label    2
dtype: int64
label    61
dtype: int64
label    54
dtype: int64


In [101]:
# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words="english",analyzer='word')

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)
count_valid = count_vectorizer.transform(X_valid)
count_tanish = count_vectorizer.transform(tanish_df["sequence"].to_numpy())

# Create the CountVectorizer DataFrame: count_train
count_train = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the CountVectorizer DataFrame: test_df
count_test = pd.DataFrame(count_test.A, columns=count_vectorizer.get_feature_names())

count_valid = pd.DataFrame(count_valid.A, columns=count_vectorizer.get_feature_names())

count_tanish = pd.DataFrame(count_tanish.A, columns=count_vectorizer.get_feature_names())

#count_tanish.head()
count_valid.head()

,00,01,0148,02,04,05,06,07,08,0843,...,yau,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english",analyzer='word')
tfidf_train= tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)
tfidf_valid = tfidf_vectorizer.transform(X_valid)
tfidf_tanish = tfidf_vectorizer.transform(tanish_df["sequence"].to_numpy())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_train  = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_test  = pd.DataFrame(tfidf_test.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_valid  = pd.DataFrame(tfidf_valid.A, columns=tfidf_vectorizer.get_feature_names())
tfidf_tanish = pd.DataFrame(tfidf_tanish.A, columns=tfidf_vectorizer.get_feature_names())

tfidf_valid

,00,01,0148,02,04,05,06,07,08,0843,...,yau,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
tfidf_train_features_df = pd.concat([tfidf_train, y_train.reset_index(drop=True)], axis=1) #ONLY run once

In [30]:
tfidf_test_features_df = pd.concat([tfidf_test, y_test.reset_index(drop = True)], axis = 1)

In [31]:
tfidf_valid_features_df = pd.concat([tfidf_valid, y_valid.reset_index(drop = True)], axis = 1)

In [83]:
tfidf_train_features_df.shape

(1053, 1212)

In [84]:
tfidf_train_features_df_new.shape

(1638, 1212)

In [38]:
tfidf_train_features_df_data = tfidf_train_features_df.values
X_smote, y_smote = tfidf_train_features_df_data[:, :-1], tfidf_train_features_df_data[:, -1]

tfidf_test_features_df_data = tfidf_test_features_df.values
X_smote_test, y_smote_test = tfidf_test_features_df_data[:, :-1], tfidf_test_features_df_data[:, -1]

tfidf_valid_features_df_data = tfidf_valid_features_df.values
X_smote_valid, y_smote_valid = tfidf_valid_features_df_data[:, :-1], tfidf_valid_features_df_data[:, -1]

In [47]:
y_smote_test

array([1., 1., 2., 2., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2., 1., 1., 2.,
       1., 1., 1., 1., 1., 1., 2., 2., 2., 1., 0., 2., 1., 1., 2., 1., 2.,
       1., 1., 2., 2., 1., 1., 1., 1., 2., 2., 1., 2., 1., 2., 0., 1., 1.,
       1., 2., 1., 1., 2., 2., 1., 1., 2., 1., 2., 2., 2., 1., 1., 2., 1.,
       1., 2., 1., 2., 2., 2., 1., 2., 2., 2., 2., 1., 0., 1., 2., 1., 2.,
       2., 1., 1., 2., 1., 2., 2., 2., 1., 2., 2., 2., 2., 2., 1., 1., 1.,
       1., 2., 1., 2., 2., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 2., 1.,
       1., 2., 1., 2., 1., 2., 1., 2., 2., 2., 2.])

In [45]:
#X_smote_test, y_smote_test = 
oversample = SMOTE()
X_train_new, y_train_new = oversample.fit_resample(X_smote, y_smote)

counter = Counter(y_train_new)
print("Train Data after SMOTE")
for k,v in counter.items():
    print('Class=%d, n=%d' % (k, v))

Train Data after SMOTE
Class=2, n=546
Class=1, n=546
Class=0, n=546


In [57]:
oversample2 = SMOTE(k_neighbors = 2)
X_test_new, y_test_new = oversample2.fit_resample(X_smote_test, y_smote_test)
counter2 = Counter(y_test_new)
print("Test Data after SMOTE")
for k,v in counter2.items():
    print('Class=%d, n=%d' % (k, v))

Test Data after SMOTE
Class=1, n=67
Class=2, n=67
Class=0, n=67


In [62]:
oversample3 = SMOTE(k_neighbors = 1)
X_valid_new, y_valid_new = oversample3.fit_resample(X_smote_valid, y_smote_valid)
counter3 = Counter(y_valid_new)
print("Test Data after SMOTE")
for k,v in counter3.items():
    print('Class=%d, n=%d' % (k, v))

Test Data after SMOTE
Class=2, n=61
Class=1, n=61
Class=0, n=61


In [67]:
X_train_new = pd.DataFrame(X_train_new)
y_train_new = pd.Series(y_train_new)

X_test_new = pd.DataFrame(X_test_new)
y_test_new = pd.Series(y_test_new)

X_valid_new = pd.DataFrame(X_valid_new)
y_valid_new = pd.Series(y_valid_new)

print(type(y_valid_new))

<class 'pandas.core.series.Series'>


In [68]:
tfidf_train_features_df_new = pd.concat([X_train_new, y_train_new.reset_index(drop=True)], axis=1)
tfidf_train_features_df_new.columns = tfidf_train_features_df.columns

tfidf_test_features_df_new = pd.concat([X_test_new, y_test_new.reset_index(drop=True)], axis=1)
tfidf_test_features_df_new.columns = tfidf_test_features_df.columns

tfidf_valid_features_df_new = pd.concat([X_valid_new, y_valid_new.reset_index(drop=True)], axis=1)
tfidf_valid_features_df_new.columns = tfidf_valid_features_df.columns

In [114]:
mappings = {0.0 : 0, 1.0 : 1, 2.0 : 2}

tfidf_train_features_df_new.label = [mappings[item] for item in tfidf_train_features_df_new.label]
tfidf_test_features_df_new.label = [mappings[item] for item in tfidf_test_features_df_new.label]
tfidf_valid_features_df_new.label = [mappings[item] for item in tfidf_valid_features_df_new.label]

# tfidf_train_features_df_new.to_csv("Data/SMOTE_train.csv")
# tfidf_test_features_df_new.to_csv("Data/SMOTE_test.csv")
# tfidf_valid_features_df_new.to_csv("Data/SMOTE_val.csv")

tfidf_features_master = pd.concat([tfidf_train_features_df_new,tfidf_test_features_df_new,tfidf_valid_features_df_new])

tfidf_features_master

,00,01,0148,02,04,05,06,07,08,0843,...,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.22719,0.0,0.0,0.000000,2
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.357893,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,2
179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,2
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,2
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,2


In [90]:
tfidf_train_features_df_new.drop(columns = 'label')
tfidf_test_features_df_new.drop(columns = 'label')
tfidf_valid_features_df_new.drop(columns = 'label')

,00,01,0148,02,04,05,06,07,08,0843,...,yau,year,years,yes,yo,young,yrs,zoloft,zyloprim,ýsocial
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
def filter_features_by_cor(df):
    m = len(df.columns)
    output = df.iloc[:,m-1] 
    output_list = output.tolist()
    corrcoef_array = []

    for i in range(0,m-2):
        input_list = df.iloc[:,i].tolist()
        cols = [input_list, output_list]
        corrcoef = abs(np.corrcoef(cols)) 
        #print(type(corrcoef))
        corrcoef_array = np.append(corrcoef_array,corrcoef[0,1])

    feature_names = list(df)
    feature_names = feature_names[0:m-2]
    output_df = pd.DataFrame(feature_names, columns=['Features'])
    output_df['CorrCoef'] = corrcoef_array
    output_df = output_df.sort_values('CorrCoef')
    output_df = output_df.reset_index()
    output_df = output_df.drop(columns = "index")
    
    return output_df

In [92]:
tfidf_output_df = filter_features_by_cor(tfidf_train_features_df_new)
tfidf_output_df = tfidf_output_df.sort_values(by=['CorrCoef'],ascending = False)
#tfidf_output_df.to_csv('Data/tfidf_vector_feature_corr.csv')

tfidf_output_df

,Features,CorrCoef
1209,accident,0.849914
1208,mellitus,0.847046
1207,cerebrovascular,0.844474
1206,reflux,0.840700
1205,gastroesophageal,0.840700
...,...,...
3,ondansetron,0.000000
1,specific,0.000000
5,333,0.000000
6,known,0.000000


In [103]:
# Setting Correlation threshold
top_tfidf_features_df = tfidf_output_df[:1000]
filtered_tfidf_train = tfidf_train_features_df_new.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_test = tfidf_test_features_df_new.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_valid   = tfidf_valid_features_df_new.filter(items=top_tfidf_features_df['Features'])
filtered_tfidf_tanish = tfidf_tanish.filter(items = top_tfidf_features_df['Features'])

In [117]:
for c in [0.01, 0.1, 1, 10, 100]:
    svm = SVC(probability=True, C=c, kernel='linear',  random_state=42)
    svm.fit(filtered_tfidf_train, y_train_new)

    scores = cross_val_score(svm, tfidf_features_master, tfidf_features_master["label"], cv=5)
    print(scores)

    y_prob_svm = svm.predict_proba(filtered_tfidf_valid)
    labels_svm = svm.predict(filtered_tfidf_valid)

    y_prob_svm_tanish = svm.predict_proba(filtered_tfidf_tanish)

tanish_svm_labels = svm.predict(filtered_tfidf_tanish)
real_tanish_labels = tanish_df["label"].to_numpy()

[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]


[1. 1. 1. 1. 1.]


In [105]:
target_names = ['NO', 'NTR', 'YES']
results_svm = classification_report(y_valid_new, labels_svm, target_names=target_names, output_dict=True)
results_svm = pd.DataFrame(results_svm).transpose()

results_svm_tanish = classification_report(real_tanish_labels, tanish_svm_labels, target_names=target_names, output_dict=True)
results_svm_tanish = pd.DataFrame(results_svm_tanish).transpose()

print(results_svm)
print("AUC: ", roc_auc_score(y_valid_new, y_prob_svm, multi_class = 'ovr'))
print("ACC: ", accuracy_score(y_valid_new, labels_svm))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid_new, labels_svm))

print("\nTanish Data: \n")
print(results_svm_tanish)
print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_svm_tanish, multi_class='ovr'))
print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_svm_labels))

              precision    recall  f1-score     support
NO             1.000000  1.000000  1.000000   61.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.983871  1.000000  0.991870   61.000000
accuracy       0.994536  0.994536  0.994536    0.994536
macro avg      0.994624  0.994536  0.994535  183.000000
weighted avg   0.994624  0.994536  0.994535  183.000000
AUC:  0.9999104183463227
ACC:  0.994535519125683

Confusion Matrix: 
 [[61  0  0]
 [ 0 60  1]
 [ 0  0 61]]

Tanish Data: 

              precision    recall  f1-score    support
NO             1.000000  0.666667  0.800000   3.000000
NTR            1.000000  0.333333  0.500000   6.000000
YES            0.722222  1.000000  0.838710  13.000000
accuracy       0.772727  0.772727  0.772727   0.772727
macro avg      0.907407  0.666667  0.712903  22.000000
weighted avg   0.835859  0.772727  0.741056  22.000000
AUC:  0.9432823511770879

Confusion Matrix: 
 [[ 2  0  1]
 [ 0  2  4]
 [ 0  0 13]]


In [65]:
for c in [0.01, 0.1, 1, 10, 100]:
    labels_lgr_tfidf = np.array([])

    lr = LogisticRegression(C=c, random_state=0)
    lr.fit(filtered_tfidf_train, y_train)
    
    y_prob_lr = lr.predict_proba(filtered_tfidf_valid)
    #print("C:", c, "\n: ", )

    labels_lgr_tfidf = lr.predict(filtered_tfidf_valid)
    
#     y_prob_lgr_tanish = lr.predict_proba(filtered_tfidf_tanish)

#     tanish_lr_labels = lr.predict(filtered_tfidf_tanish)
#     real_tanish_labels = tanish_df["label"].to_numpy()

#     results_lr_tanish = classification_report(real_tanish_labels, tanish_lr_labels, target_names=target_names, output_dict=True)
#     results_lr_tanish = pd.DataFrame(results_lr_tanish).transpose()

    results_lgr = classification_report(y_valid, labels_lgr_tfidf, target_names=target_names, output_dict=True)
    results_lgr = pd.DataFrame(results_lgr).transpose()
    
    print("\nC: ", c, "\n", results_lgr)
    print("AUC: ", roc_auc_score(y_valid, y_prob_lr, multi_class='ovr'))
    print("ACC: ", accuracy_score(y_valid, labels_lgr_tfidf))
    #print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
    print("\nConfusion Matrix: \n", confusion_matrix(y_valid, labels_lgr_tfidf))
#     print("\nTanish Data: \n", )
#     print(results_lr_tanish)
#     print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_lgr_tanish, multi_class='ovr'))
#     print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_lr_labels))
    print("---------------------------------------------------------------------\n")


C:  0.01 
               precision    recall  f1-score     support
NO             0.000000  0.000000  0.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.947368  1.000000  0.972973   54.000000
accuracy       0.974359  0.974359  0.974359    0.974359
macro avg      0.649123  0.661202  0.654903  117.000000
weighted avg   0.958614  0.974359  0.966123  117.000000
AUC:  1.0
ACC:  0.9743589743589743

Confusion Matrix: 
 [[ 0  0  2]
 [ 0 60  1]
 [ 0  0 54]]
---------------------------------------------------------------------


C:  0.1 
               precision    recall  f1-score     support
NO             0.000000  0.000000  0.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.947368  1.000000  0.972973   54.000000
accuracy       0.974359  0.974359  0.974359    0.974359
macro avg      0.649123  0.661202  0.654903  117.000000
weighted avg   0.958614  0.974359  0.966123  117.000000
AUC:  1.0
ACC:  0.97435897435

In [66]:
clf_random_forest = RandomForestClassifier()
clf_random_forest.fit(filtered_tfidf_train, y_train)

y_prob_rf = clf_random_forest.predict_proba(filtered_tfidf_valid)

labels_rf_tfidf = clf_random_forest.predict(filtered_tfidf_valid)

results_rf = classification_report(y_valid, labels_rf_tfidf, target_names=target_names, output_dict=True)
results_rf = pd.DataFrame(results_rf).transpose()
print(results_rf)
print("AUC: ", roc_auc_score(y_valid, y_prob_rf, multi_class='ovr'))
print("ACC: ", accuracy_score(y_valid, labels_rf_tfidf))
#print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
print("\nConfusion Matrix: \n", confusion_matrix(y_valid, labels_rf_tfidf))

# print("\nTanish Data: \n")

# y_prob_rf_tanish = clf_random_forest.predict_proba(filtered_tfidf_tanish)

# tanish_rf_labels = clf_random_forest.predict(filtered_tfidf_tanish)
# real_tanish_labels = tanish_df["label"].to_numpy()

# results_rf_tanish = classification_report(real_tanish_labels, tanish_rf_labels, target_names=target_names, output_dict=True)
# results_rf_tanish = pd.DataFrame(results_rf_tanish).transpose()
# print(results_rf_tanish)

# print("AUC: ", roc_auc_score(real_tanish_labels, y_prob_rf_tanish, multi_class='ovr'))
# print()
# #print("MCC: ", matthews_corrcoef(y_valid, y_prob_lr))
# print("\nConfusion Matrix: \n", confusion_matrix(real_tanish_labels, tanish_rf_labels))

              precision    recall  f1-score     support
NO             1.000000  1.000000  1.000000    2.000000
NTR            1.000000  0.983607  0.991736   61.000000
YES            0.981818  1.000000  0.990826   54.000000
accuracy       0.991453  0.991453  0.991453    0.991453
macro avg      0.993939  0.994536  0.994187  117.000000
weighted avg   0.991608  0.991453  0.991457  117.000000
AUC:  1.0
ACC:  0.9914529914529915

Confusion Matrix: 
 [[ 2  0  0]
 [ 0 60  1]
 [ 0  0 54]]
